# ICP-RANSAC
20220707 Charlotte (Shiqi WANG)

## Shared module with ICP (Same as original)

In [ ]:
def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nx3 array of points
        dst: Nx3 array of points
    Output:
        distances: Euclidean distances (errors) of the nearest neighbor
        indecies: dst indecies of the nearest neighbor
    '''

    indecies = np.zeros(src.shape[0], dtype=np.int)
    distances = np.zeros(src.shape[0])
    for i, s in enumerate(src):
        min_dist = np.inf
        for j, d in enumerate(dst):
            dist = np.linalg.norm(s-d)
            if dist < min_dist:
                min_dist = dist
                indecies[i] = j
                distances[i] = dist
    return distances, indecies  

def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform between corresponding 3D points A->B
    Input:
      A: Nx3 numpy array of corresponding 3D points
      B: Nx3 numpy array of corresponding 3D points
    Returns:
      T: 4x4 homogeneous transformation matrix
      R: 3x3 rotation matrix
      t: 3x1 column vector
    '''

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0) 
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    W = np.dot(BB.T, AA)
    U, s, VT = np.linalg.svd(W, full_matrices=True, compute_uv=True)
    R = np.dot(U, VT)

    # special reflection case
    if np.linalg.det(R) < 0:
        VT[2,:] *= -1
        R = np.dot(U, VT)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = t

    return T, R, t
       

## Module of RANSAC

In [ ]:
##RANSAC modules #c
def get_random_sample(data,n_items: int) -> np.ndarray:
    indices = np.random.choice(len(data), n_items, replace=False)
    return indices

def estimate_n_trials_needed(n_sample_points, inlier_ratio: float = 0.8, probability: float = 0.95) -> int:
    """
    Estimate number of trials needed to select only true inliers with RANSAC.
   
    Parameters
    ----------
    inlier_ratio : number of inliers / number of all points
    n_sample_points :number of selected point in each iteration
    probability : desired probability of success.
    
    Returns
    -------
    Estimated number of trials needed.
    """
    return int(np.log(1 - probability) / np.log(1 - inlier_ratio ** n_sample_points))

#main
def ransac_remove_outliers(
    observation,reference,inlier_ratio=0.8,error_threshold=10,n_sample_points=5,
    inlier_threshold=2,fit_with_final_inliers=True
)->np.ndarray: 
    '''
    This is to remove outliers after each iteration of ICP

    Parameters
    ----------
    observation - a set of points need to be registered to the ref
    inlier_ratio : number of inliers / number of all point
    error_threshold – threshold value to determine when a data point fits a model
    n_sample_points- number of selected point in each iteration
    fit_with_final_inliers: Fit final model with final inliers solved by the algorithm.
    inlier_threshold – minimum number of data points required to fit the model
  
    max_trials – maximum number of iterations allowed in the algorithm
    
    #not set # d – number of close data points required to assert that a model fits well to data
    
    Returns
    -------
    inlier indices
    '''
    # initialization
    inlier_indices_candidate = None
    inlier_indices = None
    trial_number = 0
    best_error = np.inf
    best_model = None
    

    is_termination = None #if iteration should be terminated.
    is_solution_valid = None #if solution valid or not.
    
    max_trials = estimate_n_trials_needed(n_sample_points,inlier_ratio)
    
    #fit
    for trial in range(max_trials):
        src=observation
        trial_number = trial
        sample = get_random_sample(src,n_sample_points) #get data 
        current_model,_,_= best_fit_transform(sample[0:3,:].T,reference[0:3,:].T)
        src= np.dot(current_model,src)
        #get inliers
        distance,_ = nearest_neighbor(src[0:3,:].T, reference[0:3,:].T)
        j=0
        for i, s in enumerate(src):
            d = distance[i]
            if d <= error_threshold:
                indices[j] = i
                j = j + 1
        inlier_indices_candidate = indices #find inliers
       
        #Verify if the current model is valid
        if (inlier_indices_candidate.size-n_sample_points) >= inlier_threshold:
            is_valid_solution = True
        if not is_valid_solution:
            continue
        
        error = np.sum(distance) / distance.size

        # Update solution if best so far
        if error < best_error:
            inlier_indices = inlier_indices_candidate.copy()
            best_error = error

    if fit_with_final_inliers:
        inliers = observation[inlier_indices]
        T_rs,_,_= best_fit_transform(inliers[0:3,:].T,reference[0:3,:].T)

    return T_rs

# ICP with RANSAC
## Input:
Reference 3D point set A and input 3D point set B
## Output:
Registration result of X


In [ ]:
def ICP_RANSAC(
    A, B, ICP_error,init_pose = None, max_iterations=200, tolerance=0.001
):
    '''
    The Iterative Closest Point with RANSAC
    Input:
        A: Nx3 numpy array of source 3D points
        B: Nx3 numpy array of destination 3D point
        ICP_error
        init_pose: 4x4 homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation
        distances: Euclidean distances (errors) of the nearest neighbor
    '''
    #  select points 
#     ICP_error = 30
    A,B,_,_,_ = colocalization(A,B,ICP_error)
    
    # make points homogeneous, copy them so as to maintain the originals
    src = np.ones((4,A.shape[0])) #(4, A.shape[0])
    dst = np.ones((4,B.shape[0]))
    src[0:3,:] = np.copy(A.T) # A.T shape (3,20)
    dst[0:3,:] = np.copy(B.T)
    
    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0
    distances_iter = np.zeros((max_iterations,1))

    for i in range(max_iterations):
        # find the nearest neighbours between the current source and destination points
        distances, indices = nearest_neighbor(src[0:3,:].T, dst[0:3,:].T)
        
        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[0:3,:].T, dst[0:3,indices].T)  
        # update the current source
        # refer to "Introduction to Robotics" Chapter2 P28. Spatial description and transformations
        src = np.dot(T, src)

        # add RANSAC to remove outliers #Shiqi Wang
        T = ransac_remove_outliers(src,dst)
        src = np.dot(T, src)

        # check error
        mean_error = np.sum(distances) / distances.size
        print(f'mean_error: {mean_error}')
        if abs(prev_error-mean_error) < tolerance:
           break
        prev_error = mean_error
        distances_iter[i] = mean_error

    fig=plt.figure(dpi=120,figsize=(2,3))
    plt.plot(distances_iter[:20])
    sns.despine() 
    plt.xlabel('Spots:'+ str(distances_iter[:20].shape[0]))
    plt.ylabel('Distance/pixel')
    ave=np.average(distances_iter[:20])
    plt.title(str(float('%.2f' % ave)))
    plt.show()
    # calculcate final tranformation
    T,_,_ = best_fit_transform(A, src[0:3,:].T)

    return T, distances